In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import tensorflow as tf
import scipy as scp
import sklearn as skl

In [2]:
road_data = pd.read_csv('Datasets/Real-Time_Road_Conditions.csv')
road_data.head()

,id,sensor_id,Location Name,Location,timestamp,voltage_y,voltage_x,voltage_ratio,air_temp_secondary,temp_surface,...,friction_code_displayed,friction_code_measured,friction_value_displayed,friction_value_measured,dirty_lens_score,grip_text,relative_humidity,air_temp_primary,air_temp_tertiary,status_code
0,452509,2,FM 2222 RD / LAKEWOOD DR,POINT (-97.7877274 30.3589134),02/25/2022 12:50:50 AM,1347,1552,0.868,0.15,0.51,...,3,3,0.7,0.70,4,GOOD,84.15,-0.03,9.30,101
1,452510,1,11TH ST / 35 SVRD,POINT (-97.7324753 30.2703285),02/25/2022 12:51:00 AM,2604,2178,1.196,0.21,0.35,...,8,8,0.7,0.70,0,GOOD,77.19,0.17,18.57,-102
2,452513,1,11TH ST / 35 SVRD,POINT (-97.7324753 30.2703285),02/25/2022 12:52:01 AM,2629,2177,1.208,0.33,0.41,...,8,29,0.7,0.33,0,GOOD,77.35,0.23,18.66,-102
3,452514,3,LAKELINE BLVD / 183 HWY SVRD,POINT (-97.8019562 30.4800491),02/25/2022 12:52:51 AM,1610,1518,1.061,-1.51,-0.97,...,3,3,0.7,0.70,4,GOOD,90.60,-1.63,9.24,101
4,452518,3,LAKELINE BLVD / 183 HWY SVRD,POINT (-97.8019562 30.4800491),02/25/2022 12:53:51 AM,1606,1510,1.063,-1.47,-0.93,...,3,3,0.7,0.70,4,GOOD,90.74,-1.63,9.21,101


In [4]:
# column names
column_names = road_data.columns
column_names

Index(['id', 'sensor_id', 'Location Name', 'Location', 'timestamp',
       'voltage_y', 'voltage_x', 'voltage_ratio', 'air_temp_secondary',
       'temp_surface', 'condition_code_displayed', 'condition_code_measured',
       'condition_text_displayed', 'condition_text_measured',
       'friction_code_displayed', 'friction_code_measured',
       'friction_value_displayed', 'friction_value_measured',
       'dirty_lens_score', 'grip_text', 'relative_humidity',
       'air_temp_primary', 'air_temp_tertiary', 'status_code'],
      dtype='object')

In [5]:
## removing id, sensor_id, location 
road_data.drop(columns=['id', 'sensor_id', 'Location'], inplace=True)

In [9]:
# column names after removal
column_names = road_data.columns
column_names

Index(['Location Name', 'timestamp', 'voltage_y', 'voltage_x', 'voltage_ratio',
       'air_temp_secondary', 'temp_surface', 'condition_code_displayed',
       'condition_code_measured', 'condition_text_displayed',
       'condition_text_measured', 'friction_code_displayed',
       'friction_code_measured', 'friction_value_displayed',
       'friction_value_measured', 'dirty_lens_score', 'grip_text',
       'relative_humidity', 'air_temp_primary', 'air_temp_tertiary',
       'status_code'],
      dtype='object')

In [8]:
road_data.head()

,Location Name,timestamp,voltage_y,voltage_x,voltage_ratio,air_temp_secondary,temp_surface,condition_code_displayed,condition_code_measured,condition_text_displayed,...,friction_code_displayed,friction_code_measured,friction_value_displayed,friction_value_measured,dirty_lens_score,grip_text,relative_humidity,air_temp_primary,air_temp_tertiary,status_code
0,FM 2222 RD / LAKEWOOD DR,02/25/2022 12:50:50 AM,1347,1552,0.868,0.15,0.51,1,1,DRY,...,3,3,0.7,0.70,4,GOOD,84.15,-0.03,9.30,101
1,11TH ST / 35 SVRD,02/25/2022 12:51:00 AM,2604,2178,1.196,0.21,0.35,1,1,DRY,...,8,8,0.7,0.70,0,GOOD,77.19,0.17,18.57,-102
2,11TH ST / 35 SVRD,02/25/2022 12:52:01 AM,2629,2177,1.208,0.33,0.41,1,1,DRY,...,8,29,0.7,0.33,0,GOOD,77.35,0.23,18.66,-102
3,LAKELINE BLVD / 183 HWY SVRD,02/25/2022 12:52:51 AM,1610,1518,1.061,-1.51,-0.97,1,1,DRY,...,3,3,0.7,0.70,4,GOOD,90.60,-1.63,9.24,101
4,LAKELINE BLVD / 183 HWY SVRD,02/25/2022 12:53:51 AM,1606,1510,1.063,-1.47,-0.93,1,1,DRY,...,3,3,0.7,0.70,4,GOOD,90.74,-1.63,9.21,101


In [52]:
## Randomly select n number of columns
# idea is to select 5, 10 and 15 columns for the data removal

import random

def get_random_columns(n, total_column_numbers): # 
    
    random_columns = []

    while(len(random_columns) != n):
        random_column_index = random.randrange(0, total_column_numbers)
        if(random_column_index in random_columns): continue
        random_columns.append(random_column_index)
    
    return random_columns

In [54]:
print(get_random_columns(5, 21))
print(get_random_columns(10, 21))
print(get_random_columns(15, 21))

[6, 18, 7, 5, 3]
[18, 3, 12, 7, 20, 16, 5, 9, 1, 2]
[9, 20, 10, 5, 8, 16, 0, 12, 19, 15, 2, 18, 6, 14, 13]
